# Colab Normal User

## 注意

このノートブックはColab課金ユーザのみ実行可能です。

無課金ユーザはアカウント停止の原因になります。

実行前に[利用規約](https://research.google.com/colaboratory/faq.html#disallowed-activities)を確認してください。

## 事前準備

ローカルPCでSSHキーを作成

```sh
# ターミナルで実行
ssh-keygen -t ed25519 -C "your_email@example.com"
# または RSA の場合
ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

# Enter キーを3回押す（デフォルト設定の場合）
# パスフレーズを設定することも可能
```

公開鍵の確認

```sh
cat ~/.ssh/id_ed25519.pub
# または
cat ~/.ssh/id_rsa.pub
出力例:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAI... your_email@example.com
```


## SSHサーバの起動

In [ ]:
# 公開鍵を取得
from google.colab import userdata
PUBLIC_KEY = userdata.get('PUBLIC_KEY')
assert PUBLIC_KEY

# ユーザーを作成（ホームディレクトリも作成）
!useradd -m -s /bin/bash devuser

# sudoグループに追加（sudo権限が必要な場合）
!usermod -aG sudo devuser

# SSH ディレクトリの作成
ssh_dir = f"/home/devuser/.ssh"
!mkdir -p {ssh_dir}
!chmod 700 {ssh_dir}

# authorized_keys に書き込み
authorized_keys_path = f"{ssh_dir}/authorized_keys"
with open(authorized_keys_path, 'w') as f:
    f.write(PUBLIC_KEY.strip())

# パーミッション設定
!chmod 600 {authorized_keys_path}
!chown -R devuser:devuser {ssh_dir}

# SSHサーバーのインストールと設定
print("📦 SSH サーバーをインストールしています...")
!apt-get update -qq
!apt-get install -y openssh-server > /dev/null 2>&1

# SSH設定ファイルを作成
sshd_config = """Port 22
PermitRootLogin no
PubkeyAuthentication yes
PasswordAuthentication no
ChallengeResponseAuthentication no
UsePAM yes
X11Forwarding yes
PrintMotd no
AcceptEnv LANG LC_*
Subsystem sftp /usr/lib/openssh/sftp-server
"""

with open('/etc/ssh/sshd_config', 'w') as f:
    f.write(sshd_config)

# SSHサービスを起動
!service ssh start
!service ssh status | grep Active

print("\n✅ SSH サーバーが起動しました")
print("   - ルートログイン: 無効")
print("   - パスワード認証: 無効")
print("   - 公開鍵認証: 有効\n")

# ユーザー情報を表示
print(f"📋 ユーザー情報:")
!id devuser

# /content へのアクセス権限を付与
!chown -R devuser:devuser /content
print(f"\n✅ /content の所有者を devuser に変更しました")

# devuser に sudo 権限を付与
!echo 'devuser ALL=(ALL) NOPASSWD: ALL' > /etc/sudoers.d/devuser
!chmod 440 /etc/sudoers.d/devuser

## 環境構築

In [ ]:
import subprocess

# .bashrcに cd /content を追加
!echo 'cd /content' >> /home/devuser/.bashrc

# Google Colab 環境変数を.bashrcに追加
!env | grep -E "^(LD_LIBRARY_PATH|PATH|CUDA|NVIDIA_VISIBLE|COLAB_GPU)=" | \
    sed 's/^/export /' >> /home/devuser/.bashrc
print("✅ 環境変数が /home/devuser/.bashrc に追加されました。\n")

# 先にbunをインストール（並列インストールで使用するため）
print("📦 bunをインストールしています...")
!su - devuser -c 'curl -fsSL https://bun.sh/install | bash'

# 並列インストールコマンド（devuserとして実行、npmパッケージはbunでまとめてインストール）
commands = [
    "su - devuser -c 'curl -fsSL https://claude.ai/install.sh | bash'",
    "su - devuser -c '$HOME/.bun/bin/bun add -g @openai/codex @google/gemini-cli @github/copilot'",
    "su - devuser -c 'curl -LsSf https://astral.sh/uv/install.sh | sh'",
    "su - devuser -c \"curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y\"",
    "curl -fsSL https://raw.githubusercontent.com/hayatoshibahara/tmux/refs/heads/main/.tmux.conf -o /home/devuser/.tmux.conf",
]

# 並列実行
processes = [subprocess.Popen(cmd, shell=True) for cmd in commands]

print("⏳ ツールをインストールしています。完了までお待ちください...\n")
for p in processes:
    p.wait()

# .tmux.confの所有者を変更
!chown devuser:devuser /home/devuser/.tmux.conf

# Python インストール（uv完了後に実行）
!su - devuser -c '$HOME/.local/bin/uv python install 3.13 --default'

# .bashrc設定
with open('/home/devuser/.bashrc', 'a') as f:
    f.write('\n# ツールのPATH\n')
    f.write('export PATH="$HOME/.local/bin:$HOME/.bun/bin:$HOME/.local/bin:$HOME/.cargo/bin:$PATH"\n')
    f.write('alias claude="claude --dangerously-skip-permissions"\n')
    f.write('export UV_PYTHON=3.13\n')
    f.write('eval "$(uv generate-shell-completion bash)"\n')
    f.write('eval "$(uvx --generate-shell-completion bash)"\n')

print("\n✅ すべてのツールがインストールされました。")

## Cloudflaredの起動

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /tmp/cloudflared
!chmod +x /tmp/cloudflared
!/tmp/cloudflared tunnel --url ssh://localhost:22

トンネルURLが「https://comparing-trim-background-improve.trycloudflare.com」の場合、以下をローカルPCで設定する。

~/.ssh/config に追加:

```sh
Host colab
HostName comparing-trim-background-improve.trycloudflare.com
User devuser
ProxyCommand cloudflared access ssh --hostname %h
IdentityFile ~/.ssh/id_ed25519
IdentitiesOnly yes
StrictHostKeyChecking no
UserKnownHostsFile /dev/null
ServerAliveInterval 60
ServerAliveCountMax 3
```

これにより `ssh colab` やVSCodeでSSH接続できる。